In [17]:
import pandas as pd
import json
from statistics import mean
import numpy as np
from pprint import pprint 
from xgboost import XGBRegressor
import copy 


### Load Data & Clean Data

In [18]:
data_source_file = "./data-frame-friendly.json"

f = open (data_source_file, "r")
data = json.load(f)
df1 = pd.DataFrame.from_dict(data, orient='index')
df1=df1.reset_index()

In [19]:
full_dict_storage = [] 
#0 player did not play 
#1 player is loser 
#2 player is winner
for index,row in df1.iterrows(): 
    
    player_names ={
    "andy":0,
    "brandon":0,
    "brian":0,
    "cade":0,
    "darwin":0,
    "josh":0,
    "lindsey":0,
    "sequential":0,
    "sophie":0,
    "steve":0,
    "steven":0,
    "sun":0,
    "susi":0,
    "susu":0,
    "tang":0,
    "yang":0,
    "andy_acs":0,
    "brandon_acs":0,
    "brian_acs":0,
    "cade_acs":0,
    "darwin_acs":0,
    "josh_acs":0,
    "lindsey_acs":0,
    "sequential_acs":0,
    "sophie_acs":0,
    "steve_acs":0,
    "steven_acs":0,
    "sun_acs":0,
    "susi_acs":0,
    "susu_acs":0,
    "tang_acs":0,
    "yang_acs":0,
    "score_diff":0}
    
    
    min_team_size = min(len(row["team_red"]),len(row["team_blue"]))
#     print(min_team_size)
    if row["score_red"] > row["score_blue"]: # this means team a won 

#         print(row["team_red"][0]["player_name"])
        for index_2 in range(0,min_team_size,1): # go from 0 to five #HUGE BUG HERE 
        
            row_data = row["team_red"][index_2]
            #doing team a player assignment
            name = row["team_red"][index_2]["player_name"]
            player_names[name] = player_names.get(name, 0) + 2
            
            # doing the other team assignment 
            name = row["team_blue"][index_2]["player_name"]
            player_names[name] = player_names.get(name, 0) + 1
            
            #adding acs data 
            name = row["team_red"][index_2]["player_name"]
            new_dict_label = str(name)+"_acs"
            player_names[new_dict_label] = player_names.get(new_dict_label, 0) + row["team_red"][index_2]["average_combat_score"]
            
            name = row["team_blue"][index_2]["player_name"]
            new_dict_label = str(name)+"_acs"
            player_names[new_dict_label] = player_names.get(new_dict_label, 0) + row["team_red"][index_2]["average_combat_score"]


    elif row["score_red"] < row["score_blue"]: # this means team b won 
        for index_2 in range(0,min_team_size,1): # go from 0 to five
            name = row["team_red"][index_2]["player_name"]
            player_names[name] = player_names.get(name, 0) + 1
            
            # doing the other team assignment 
            name = row["team_blue"][index_2]["player_name"]
            player_names[name] = player_names.get(name, 0) + 2
            
            #adding acs data 
            name = row["team_red"][index_2]["player_name"]
            new_dict_label = str(name)+"_acs"
            player_names[new_dict_label] = player_names.get(new_dict_label, 0) + row["team_red"][index_2]["average_combat_score"]
            
            name = row["team_blue"][index_2]["player_name"]
            new_dict_label = str(name)+"_acs"
            player_names[new_dict_label] = player_names.get(new_dict_label, 0) + row["team_red"][index_2]["average_combat_score"]

    player_names['score_diff'] = abs(row["score_red"] - row["score_blue"]) # i think i take abs value
#     player_names['score_diff'] = row["score_red"] - row["score_blue"]
    print(player_names) # UNCOMMENT THIS TO SHOW RESULTS 
    
    full_dict_storage.append(player_names)
#     break

{'andy': 0, 'brandon': 0, 'brian': 1, 'cade': 2, 'darwin': 2, 'josh': 1, 'lindsey': 1, 'sequential': 0, 'sophie': 2, 'steve': 1, 'steven': 1, 'sun': 2, 'susi': 0, 'susu': 0, 'tang': 2, 'yang': 0, 'andy_acs': 0, 'brandon_acs': 0, 'brian_acs': 367, 'cade_acs': 141, 'darwin_acs': 264, 'josh_acs': 141, 'lindsey_acs': 264, 'sequential_acs': 0, 'sophie_acs': 139, 'steve_acs': 350, 'steven_acs': 139, 'sun_acs': 350, 'susi_acs': 0, 'susu_acs': 0, 'tang_acs': 367, 'yang_acs': 0, 'score_diff': 9}
{'andy': 0, 'brandon': 0, 'brian': 1, 'cade': 1, 'darwin': 1, 'josh': 1, 'lindsey': 2, 'sequential': 0, 'sophie': 2, 'steve': 2, 'steven': 2, 'sun': 1, 'susi': 0, 'susu': 0, 'tang': 2, 'yang': 0, 'andy_acs': 0, 'brandon_acs': 0, 'brian_acs': 21, 'cade_acs': 180, 'darwin_acs': 118, 'josh_acs': 194, 'lindsey_acs': 194, 'sequential_acs': 0, 'sophie_acs': 180, 'steve_acs': 21, 'steven_acs': 118, 'sun_acs': 279, 'susi_acs': 0, 'susu_acs': 0, 'tang_acs': 279, 'yang_acs': 0, 'score_diff': 6}
{'andy': 2, 'brand

In [20]:
df_generated = pd.DataFrame.from_dict(full_dict_storage)
df_generated = df_generated.replace(np.nan,0)
# df_generated # this is the full dataset used for training 

In [21]:
# select the x_train columns 
x_train= df_generated[["andy",
    "brandon",
    "brian",
    "cade",
    "darwin",
    "josh",
    "lindsey",
    "sequential",
    "sophie",
    "steve",
    "steven",
    "sun",
    "susi",
    "susu",
    "tang",
    "yang",
    "andy_acs",
    "brandon_acs",
    "brian_acs",
    "cade_acs",
    "darwin_acs",
    "josh_acs",
    "lindsey_acs",
    "sequential_acs",
    "sophie_acs",
    "steve_acs",
    "steven_acs",
    "sun_acs",
    "susi_acs",
    "susu_acs",
    "tang_acs",
    "yang_acs"]]

display(x_train)
# x_train = df_generated.iloc[:,:16]
x_train = x_train.to_numpy()
# move column position  https://www.geeksforgeeks.org/how-to-move-a-column-to-first-position-in-pandas-dataframe/
y_train = df_generated[["score_diff"]]
y_train = y_train.to_numpy()

# y_train

,andy,brandon,brian,cade,darwin,josh,lindsey,sequential,sophie,steve,...,lindsey_acs,sequential_acs,sophie_acs,steve_acs,steven_acs,sun_acs,susi_acs,susu_acs,tang_acs,yang_acs
0,0,0,1,2,2,1,1,0,2,1,...,264,0,139,350,139,350,0,0,367,0
1,0,0,1,1,1,1,2,0,2,2,...,194,0,180,21,118,279,0,0,279,0
2,2,0,1,2,0,2,1,0,2,0,...,244,0,193,0,151,0,0,0,151,0
3,1,0,1,2,2,1,0,0,2,0,...,0,0,138,0,0,439,138,239,0,0
4,1,0,2,1,2,1,0,2,1,0,...,0,225,86,0,0,181,290,86,0,0
5,2,0,1,1,2,2,1,1,2,0,...,312,240,240,0,0,0,129,0,129,0
6,2,0,2,2,2,1,1,1,1,0,...,304,139,382,0,0,0,279,0,304,0
7,1,0,0,1,2,1,1,2,1,0,...,224,335,239,0,0,0,169,0,224,0
8,1,0,1,2,1,2,2,0,2,0,...,200,0,122,0,200,271,0,0,0,0
9,0,0,0,2,2,2,2,1,1,0,...,253,253,69,0,0,0,273,246,171,0


In [22]:
x_train

array([[  0,   0,   1, ...,   0, 367,   0],
       [  0,   0,   1, ...,   0, 279,   0],
       [  2,   0,   1, ...,   0, 151,   0],
       ...,
       [  2,   0,   1, ...,   0,   0,   0],
       [  2,   0,   1, ...,   0,   0,   0],
       [  1,   0,   1, ...,   0, 384,   0]], dtype=int64)

### Create XG Regressor Model 

In [23]:
xgb_model = XGBRegressor()
# model = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)# try linear parameter and other stuff 
# paramenters can be better tuned 

# learning rate will affect the results!!!
xgb_model = XGBRegressor(nthread=-1, seed=500, eval_metric="rmse", n_estimators=300,learning_rate=1)

xgb_model.fit(x_train,y_train)


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric='rmse', feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=1, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=300, n_jobs=-1, nthread=-1,
             num_parallel_tree=1, predictor='auto', ...)

In [24]:
import itertools
# this is where you input the player names and map 
players = ["andy","darwin","steve","steven","sophie","brandon","lindsey","josh","susu"] # 10 players
map_name = "Icebox"
print(len(players)) # players should be 10 

all_combinations = []
for comb in itertools.combinations(players, 5):
    both_combs = {} 
    
    comb = list(comb) # first combination
    both_combs[1] = comb
    
    # run every possible combination of 10 choose 5 
    other_comb = list(set(players)-set(comb))# other combination 
    both_combs[2] = other_comb
    
#     print(other_comb)
    all_combinations.append(both_combs) 
# all_combinations

9


### Predict player ACS (based on average of 3 maps + average of 3 recent games) 

In [25]:
 #https://www.geeksforgeeks.org/read-json-file-using-python/
# data_source_file = "./data copy.json"
data_source_file = "./data-frame-friendly.json"

f = open (data_source_file, "r")
data = json.load(f)
df1 = pd.DataFrame.from_dict(data, orient='index')
df1=df1.reset_index()

### we flatten the df here 
row_storage = []

for main_index,main_row in df1.iterrows(): 
#     print(main_index)
    meta_data = df1[['time','url','map','score_red','score_blue']].iloc[main_index]

    teams = ["team_red","team_blue"]

    for team_number in teams:
        df2 = pd.json_normalize(df1[team_number].iloc[main_index]) #take the first row 


        for index,row in df2.iterrows():
            new_row = pd.concat([meta_data, row])
            row_storage.append(new_row)


new_df = pd.DataFrame(row_storage)


### predict player acs 
def calc_player_acs_3(player_name,map_name): # we are taking the top
#     player_name = "brian"
    # def calc_player_average

    recent3_maps = new_df.loc[(new_df['player_name']==player_name)&(new_df['map']==map_name)]['average_combat_score'][-10:].tolist()
    recent3_games = new_df.loc[(new_df['player_name']==player_name)]['average_combat_score'][-10:].tolist()
    
#     print(recent3_maps)
#     print(recent3_games)
    
    # we take the mean of these 6 values
    recent_6_sum = recent3_maps + recent3_games
    average_acs = sum(recent_6_sum) /(len(recent3_maps) + len(recent3_games))

    
    return average_acs

test_one = calc_player_acs_3('susu','Icebox')
# print(test_one)


In [ ]:
player_x_predict = {
    "andy":0,
    "brandon":0,
    "brian":0,
    "cade":0,
    "darwin":0,
    "josh":0,
    "lindsey":0,
    "sequential":0,
    "sophie":0,
    "steve":0,
    "steven":0,
    "sun":0,
    "susi":0,
    "susu":0,
    "tang":0,
    "yang":0,
    "andy_acs":0,
    "brandon_acs":0,
    "brian_acs":0,
    "cade_acs":0,
    "darwin_acs":0,
    "josh_acs":0,
    "lindsey_acs":0,
    "sequential_acs":0,
    "sophie_acs":0,
    "steve_acs":0,
    "steven_acs":0,
    "sun_acs":0,
    "susi_acs":0,
    "susu_acs":0,
    "tang_acs":0,
    "yang_acs":0}

for p_name in players: 
    acs_average = calc_player_acs_3(p_name,map_name)

    key_name = p_name + "_acs" # we are "predicting" the player's acs by taking their average 
    player_x_predict[key_name] = player_x_predict.get(key_name, 0) + acs_average
    
    
print(player_x_predict)
        

### Predict score difference of every combination of players in 2 teams (10 choose 5)

In [26]:
all_combs_storage =[]

# this step is creating each x_train from each combintation 
for i in all_combinations:
    # this is the first 252 combinations (where we think 2 will be the winning team as trained in the model)

    player_names = copy.deepcopy(player_x_predict) #need this to copy the acs per person

    for name in i[1]: # person who is 2 matters more 
        player_names[name] = player_names.get(name, 0) + 1 #assign to team 1

    for name in i[2]: 
        player_names[name] = player_names.get(name, 0) + 2 #assign to team 2
    
    y_data= list(player_names.values())
    y_predict = xgb_model.predict([y_data])
    
#     print(player_names.keys(),y_predict)
    y_predict = float(y_predict)
    answer = {} 
    answer[y_predict]= player_names
    all_combs_storage.append(answer)
    
    #-----------------------------------------------------------------------------
    #now we have to do the next 252 combinations and flip 1 and 2s (2s will be the winning team)
    player_names = copy.deepcopy(player_x_predict) #need this to copy the acs per person

    for name in i[1]: # person who is 2 matters more 
        player_names[name] = player_names.get(name, 0) + 1 #assign to team 1

    for name in i[2]: 
        player_names[name] = player_names.get(name, 0) + 2 #assign to team 2
        
    y_data= list(player_names.values())
    y_predict = xgb_model.predict([y_data])
    
#     print(player_names.keys(),y_predict)
    y_predict = float(y_predict)
    answer = {} 
    answer[y_predict]= player_names
    all_combs_storage.append(answer)
    
    
print(len(all_combs_storage))
pprint(all_combs_storage[:10])

252
[{3.2935826778411865: {'andy': 1,
                       'andy_acs': 297.8,
                       'brandon': 2,
                       'brandon_acs': 110.1,
                       'brian': 0,
                       'brian_acs': 0,
                       'cade': 0,
                       'cade_acs': 0,
                       'darwin': 1,
                       'darwin_acs': 234.0,
                       'josh': 2,
                       'josh_acs': 176.625,
                       'lindsey': 2,
                       'lindsey_acs': 205.6875,
                       'sequential': 0,
                       'sequential_acs': 0,
                       'sophie': 1,
                       'sophie_acs': 151.25,
                       'steve': 1,
                       'steve_acs': 214.5,
                       'steven': 1,
                       'steven_acs': 242.69230769230768,
                       'sun': 0,
                       'sun_acs': 0,
                       'susi': 0,
         

### Find the smallest predicted score differences in all combinations

In [12]:
final_answer = [] 
smallest_value = 99
for i in all_combs_storage:
    for k, v in i.items():
        if float(k) <smallest_value: 
            smallest_value = float(k)
            final_answer = i  
            
### Find players who were assigned to team 1 and team 2 

team_1 = [] 
team_2 = [] 
for key,values in final_answer.items(): 
    counter = 0 # stop after 17
    for keys2,values2 in values.items(): 
#         print(keys2,values2)
        
        if counter <= 17: #there are 17 players
            if values2 == 1: 
                team_1.append(keys2)
            elif values2 == 2: 
                team_2.append(keys2)

            counter = counter + 1 
            
print('Predicted score difference:',smallest_value)            
print(team_1)
print(team_2)

Predicted score difference: 1.7343651056289673
['brandon', 'lindsey', 'sophie', 'steve', 'steven']
['andy', 'darwin', 'josh', 'susu']


## Try pick order by acs 

In [13]:
first_key = [float(i) for i in final_answer.keys()][0]
final_answer = final_answer.pop(first_key)
# Using items() + list slicing 

# Get first K items in dictionary 
final_answer_sliced = dict(list(final_answer.items())[16:]) # THERE ARE ONLY 16 PLAYERS  



In [14]:
# ### add new player here 
final_answer_sliced['brandon_brother'] = 150

# need to sort from greatest to least
final_answer_sliced=dict(sorted(final_answer_sliced.items(), key=lambda x: -x[1]))

print(final_answer_sliced)

{'andy_acs': 297.8, 'steven_acs': 242.69230769230768, 'darwin_acs': 234.0, 'susu_acs': 221.375, 'steve_acs': 214.5, 'lindsey_acs': 205.6875, 'josh_acs': 176.625, 'sophie_acs': 151.25, 'brandon_brother': 150, 'brandon_acs': 110.1, 'brian_acs': 0, 'cade_acs': 0, 'sequential_acs': 0, 'sun_acs': 0, 'susi_acs': 0, 'tang_acs': 0, 'yang_acs': 0}


In [15]:
final_answer_sliced

{'andy_acs': 297.8,
 'steven_acs': 242.69230769230768,
 'darwin_acs': 234.0,
 'susu_acs': 221.375,
 'steve_acs': 214.5,
 'lindsey_acs': 205.6875,
 'josh_acs': 176.625,
 'sophie_acs': 151.25,
 'brandon_brother': 150,
 'brandon_acs': 110.1,
 'brian_acs': 0,
 'cade_acs': 0,
 'sequential_acs': 0,
 'sun_acs': 0,
 'susi_acs': 0,
 'tang_acs': 0,
 'yang_acs': 0}

In [16]:
team_1_players = []
team_2_players = []
# ABBAABBABA
team_1_order = [0,3,4,7,9]
team_2_order = [1,2,5,6,8] 

# brandon pick order
# team_1_order = [0,3,4,7,8]
# team_2_order = [1,2,5,6,9] 

# find optimal pick order.... 
counter = 0 
for keys,values in final_answer_sliced.items():
    
    if counter <=9: 
        index = index = list(final_answer_sliced).index(keys) #keys = "andy" # get the index per key 
        if index in team_1_order: 
            team_1_players.append(keys)
#             print(keys)
        else: 
            team_2_players.append(keys)
#             print('2',keys)
    #     print(keys,values)
    counter += 1 
    
print(team_1_players,team_2_players)

['andy_acs', 'susu_acs', 'steve_acs', 'sophie_acs', 'brandon_brother'] ['steven_acs', 'darwin_acs', 'lindsey_acs', 'josh_acs', 'brandon_acs']
